# Project Gutenberg Cooking Docs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob 
import re             
import nltk
import os

In [48]:
os.chdir('C:/Users/Jason/Documents/Data Science/Spring 2020/Text Analytics/final_project/DS5001-Text-Analytics/Cookbooks/Various')

In [3]:
%matplotlib inline
sns.set()

In [4]:
OHCO = ['book_id', 'sect_num', 'para_num', 'sent_num', 'token_num']

In [5]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

# Define Functions

### Creating the library

In [62]:
file_dir = 'C:/Users/Jason/Documents/Data Science/Spring 2020/Text Analytics/final_project/DS5001-Text-Analytics/Cookbooks/Various'
epub_list = []
for d in os.listdir(file_dir):
    if '.txt' in d:
        epub_list.append(d)
epub_list

['Boland1893_Invalid_pg54568.txt',
 'Bradley1923_FLSG_pg10582.txt',
 'Bury1844_TLOCB_pg29232.txt',
 'deSalis1888_Game_pg31982.txt',
 'Eaton1822_TCHCUD_pg29084.txt',
 'Francatelli1852_PCBWC_pg22114.txt',
 'Goudiss1918_War_pg15464.txt',
 'Hill1909_SSCDD_pg19077.txt',
 'Hiller1913_FSD_pg31534.txt',
 'Kitchiner1830_TCO_pg28681.txt',
 'Kleber1915_Suffrage_pg26323.txt',
 'Leslie1840_DFC_pg9624.txt',
 'Lusk1918_War_pg32472.txt',
 'Murrey1884_Soups_pg25007.txt',
 'Murrey1885_Bfast_pg24205.txt',
 'Murrey1885_Salads_pg24542.txt',
 'Murrey1888_Fish_pg53521.txt',
 'Murrey1891_Curry_pg38464.txt',
 'Nelsons1892_HC_pg29519.txt',
 'Payne1877_CSPC_pg61185.txt',
 'Randolf1860_VAHousewife_pg12519.txt',
 'Reed1916_TMRCB_pg37680.txt',
 'Ronald1901_Century_pg34822.txt',
 'Santiagoe1887_Curry_pg34107.txt',
 'Various1902_LuncheonDishes_pg24384.txt',
 'Various1913_AMCM_pg26005.txt',
 'WIDAS1923_WILCV01_pg9935.txt',
 'WIDAS1923_WILCV02_pg9936.txt',
 'WIDAS1923_WILCV03_pg9937.txt',
 'WIDAS1923_WILCV04_pg9938.txt

In [69]:
def create_library(epub_list):
    my_lib = []
    for epub_file in epub_list:
        book_id = int(epub_file.split('_pg')[-1].split('.')[0])
        author_yr = epub_file.split('_')[0].replace('[0-9]','') 
        author_last = author_yr[:-4] #isolating the author's last name
        book_year = int(author_yr[-4:]) #isolating the book year
        
        my_lib.append((book_id, author_last, book_year, epub_file))
        
    library = pd.DataFrame(my_lib, columns=['book_id', 'author_last', 'book_year', 'book_file']).set_index('book_id')
    
    return library


In [70]:
library = create_library(epub_list)
library

,author_last,book_year,book_file
book_id,,,
54568,Boland,1893,Boland1893_Invalid_pg54568.txt
10582,Bradley,1923,Bradley1923_FLSG_pg10582.txt
29232,Bury,1844,Bury1844_TLOCB_pg29232.txt
31982,deSalis,1888,deSalis1888_Game_pg31982.txt
29084,Eaton,1822,Eaton1822_TCHCUD_pg29084.txt
22114,Francatelli,1852,Francatelli1852_PCBWC_pg22114.txt
15464,Goudiss,1918,Goudiss1918_War_pg15464.txt
19077,Hill,1909,Hill1909_SSCDD_pg19077.txt
31534,Hiller,1913,Hiller1913_FSD_pg31534.txt


## Creating the documents tables

Because of differences in styles, groups of cookbooks needed to hav their document tables created separately

### WIDAS cookbooks books

(Widas cookbooks were a group of related cookbooks published in 1923)

#### Getting the WIDAS cookbooks only from the directory

In [8]:
file_dir = 'C:/Users/Jason/Documents/Data Science/Spring 2020/Text Analytics/final_project/DS5001-Text-Analytics/Cookbooks/Various'
WIDAS_list = []
for d in os.listdir(file_dir):
    if 'WIDAS' in d:
        WIDAS_list.append(d)
WIDAS_list

['WIDAS1923_WILCV01_pg9935.txt',
 'WIDAS1923_WILCV02_pg9936.txt',
 'WIDAS1923_WILCV03_pg9937.txt',
 'WIDAS1923_WILCV04_pg9938.txt',
 'WIDAS1923_WILCV05_pg9939.txt']

In [76]:
def get_WIDAS_doc(WIDAS, OHCO=OHCO):
    
    my_doc = []

    for epub_file in WIDAS:

        book_id = int(epub_file.split('_pg')[-1].split('.')[0].replace('b','1'))

        # Import file as lines
        lines = open(epub_file, 'r', encoding='utf-8-sig').readlines()
        df = pd.DataFrame(lines, columns=['line_str'])
        df.index.name = 'line_num'
        df.line_str = df.line_str.str.strip()
        df['book_id'] = str(book_id)

        # Remove cruft from front and back
        start = df[df.line_str.str.match(re.compile('\*\*\*\s*START'), 1)].index[0]   
        stop  = df[df.line_str.str.match(re.compile('(\*\*\*)*End of (the )*Project Gutenberg', re.IGNORECASE), 1)].index[0]
        #print('BOOK_ID', book_id, start, stop)
        df = df.iloc[(start+1):stop]
        
        # Chunk by section
        sect = re.compile('\*\s\s\s\s')
        sect_lines = df.line_str.str.match(sect)
        sect_nums = [i+1 for i in range(df.loc[sect_lines].shape[0])]
        print('sect_nums: ', sect_nums, '')
        #print(df.loc[sect_lines].head(), '\n')
        df.loc[sect_lines, 'sect_num'] = sect_nums
        df.sect_num = df.sect_num.ffill()

        # Clean up
        df = df[~df.sect_num.isna()] # Remove chapter heading lines
        df = df.loc[~sect_lines]     # Remove everything before Section 1
        df['sect_num'] = df['sect_num'].astype('int')
        
        # Group -- Note that we exclude the book level in the OHCO at this point
        df = df.groupby(OHCO[1:2]).line_str.apply(lambda x: '\n'.join(x)).to_frame() # Make big string

        # Split into paragrpahs
        df = df['line_str'].str.split(r'\n\n+', expand=True).stack().to_frame().rename(columns={0:'para_str'})
        df.index.names = OHCO[1:3] # MAY NOT BE NECESSARY UNTIL THE END
        df['para_str'] = df['para_str'].str.replace(r'\n', ' ').str.strip()
        df = df[~df['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs

        # Set index
        df['book_id'] = book_id
        df = df.reset_index().set_index(OHCO[:3])

        # Register
        my_doc.append(df)    
    
    docs = pd.concat(my_doc)
    print("Done.")
    return docs   

In [77]:
docs_widas = get_WIDAS_doc(WIDAS_list)

sect_nums:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40] 
sect_nums:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21] 
sect_nums:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27] 
sect_nums:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27] 
sect_nums:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51] 
Done.


In [78]:
docs_widas.sample(20)

para_str
book_id sect_num para_num                                                   
9939    5        34                                              RASPBERRIES
9938    4        71        139. If sandwiches that are entirely different...
        2        1         45. With the knowledge already obtained of the...
9937    19       22                                    EXAMINATION QUESTIONS
9939    22       49                                            PLUM CONSERVE
9936    18       42                 LIMA-BEAN LOAF (Sufficient to Serve Six)
9935    23       42        SPAGHETTI WITH CHEESE AND TOMATO SAUCE (Suffic...
9939    50       15        (14) What can be done to balance the cost of f...
9936    9        48        53. CHEESE WAFERS.--If made daintily, cheese w...
9935    20       74        65. Another satisfactory dish may be made by c...
9937    2        8                          STOCK FOR CLEAR SOUP OR BOUILLON
9938    11       105       Crush the strawberries, add them with the juic...
9939    15       63        The advantage of this classification, as in th...
9937    11       0                                                      PORK
9936    18       15                           SAUCE FOR CREAMED STRING BEANS
9939    19       6         Besides their importance in food value, they s...
9935    12       21        1 Tb. corn starch 3/4 c. boiling water 2 Tb. b...
        14       7         Orange Fluff: stiff enough? too soft? flavor a...
9939    37       19        54. AFTERNOON TEA.--When tea is desired for af...
        47       9         In a rough way, the various occupations for bo...

### Get Murrey cookbooks


(Murrey cookbooks were a group of related cookbooks published in the late 1800s)

In [44]:
file_dir = 'C:/Users/Jason/Documents/Data Science/Spring 2020/Text Analytics/final_project/DS5001-Text-Analytics/Cookbooks/Various'
murrey_list = []
for d in os.listdir(file_dir):
    if 'Murrey' in d:
        murrey_list.append(d)
murrey_list

['Murrey1884_Soups_pg25007.txt',
 'Murrey1885_Bfast_pg24205.txt',
 'Murrey1885_Salads_pg24542.txt',
 'Murrey1888_Fish_pg53521.txt',
 'Murrey1891_Curry_pg38464.txt']

In [45]:
def get_Murrey_doc(murrey_list=murrey_list, OHCO=OHCO):
    
    my_doc = []

    for epub_file in murrey_list:

        book_id = int(epub_file.split('_pg')[-1].split('.')[0].replace('b','1'))

        # Import file as lines
        lines = open(epub_file, 'r', encoding='utf-8-sig').readlines()
        df = pd.DataFrame(lines, columns=['line_str'])
        df.index.name = 'line_num'
        df.line_str = df.line_str.str.strip()
        df['book_id'] = str(book_id)

        # Remove cruft from front and back
        start = df[df.line_str.str.match(re.compile('\*\*\*\s*START'), 1)].index[0]   
        stop  = df[df.line_str.str.match(re.compile('(\*\*\*)*End of (the )*Project Gutenberg', re.IGNORECASE), 1)].index[0]
        #print('BOOK_ID', book_id, start, stop)
        df = df.iloc[(start+1):stop].reset_index(drop=True)
        
        # Chunk by section
        test_list = []
        for i in range(len(df)):
            try:
                if df.iloc[i,0]=='' and df.iloc[i+1,0]==''and df.iloc[i+2,0]=='' and df.iloc[i+3,0]=='':
                    value = True
                else:
                    value = False
                test_list.append(value)
            except:
                value = False
                test_list.append(value)
        sect_lines = pd.Series(test_list)
        sect_nums = [i+1 for i in range(df.loc[sect_lines].shape[0])]
        print('sect_nums: ', sect_nums, '')
        #print(df.loc[sect_lines].head(), '\n')
        df.loc[sect_lines, 'sect_num'] = sect_nums
        df.sect_num = df.sect_num.ffill()
                
        # Clean up
        df = df[~df.sect_num.isna()] # Remove chapter heading lines
        df = df.loc[~sect_lines]     # Remove everything before Section 1
        df['sect_num'] = df['sect_num'].astype('int')
                
        # Group -- Note that we exclude the book level in the OHCO at this point
        df = df.groupby(OHCO[1:2]).line_str.apply(lambda x: '\n'.join(x)).to_frame() # Make big string

        # Split into paragrpahs
        df = df['line_str'].str.split(r'\n\n+', expand=True).stack().to_frame().rename(columns={0:'para_str'})
        df.index.names = OHCO[1:3] # MAY NOT BE NECESSARY UNTIL THE END
        df['para_str'] = df['para_str'].str.replace(r'\n', ' ').str.strip()
        df = df[~df['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs

        # Set index
        df['book_id'] = book_id
        df = df.reset_index().set_index(OHCO[:3])

        # Register
        my_doc.append(df)    
    
    docs = pd.concat(my_doc)
    print("Done.")
    return docs   

In [71]:
docs_murrey = get_Murrey_doc()
docs_murrey.sample(20)

sect_nums:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 
sect_nums:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21] 
sect_nums:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16] 
sect_nums:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26] 
sect_nums:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78] 
Done.


para_str
book_id sect_num para_num                                                   
24542   12       16        _12mo., 128 pages, cloth, neat stamping in gol...
53521   22       6         =New-England Codfish Balls.=--Shred the codfis...
        8        158       BROILED SMELTS, SAUCE TARTARE                 ...
        2        4         Author of “Fifty Soups,” “Fifty Salads,” “Brea...
25007   5        35         MARROW DUMPLINGS,                             11
53521   21       2         =Salmon Steak.=--Put into fast boiling water, ...
        11       6         The average New Yorker may call the City of Br...
24205   1        1         Produced by Annie McGuire and the Online Distr...
53521   22       8         Soak in cold water until soft a sufficiency of...
24542   10       19        CAULIFLOWER SALAD.--Put into a basin of cold w...
24205   9        1                                               DEDICATION.
38464   27       1                                           Curry of Frogs.
53521   8        24        PICKLED-OYSTER OMELET                         ...
24205   15       7         =Milk Toast.=--Wet the pan to be used with col...
24542   10       10        ANCHOVY SALAD.--Wash, skin, and bone eight sal...
53521   12       4         =Soft Clams in Chafing-Dish.=--Select a dozen ...
24205   12       5         =Blackberries, Raspberries, Whortleberries=, e...
53521   8        87        TO SERVE OYSTER CRABS                         ...
                 98        PRAWNS, SAUTÉ, À LA MARENGO                   ...
        6        1                                            [Illustration]

In [82]:
doc_j = pd.concat([docs_murrey, docs_widas])
doc_j.sample(20)

para_str
book_id sect_num para_num                                                   
9937    23       34        33. STUFFING FOR FISH.--As has been mentioned,...
53521   8        146       CURRY OF SHAD ROE                             ...
9936    20       270                           TURNIPS AND THEIR PREPARATION
                 52        24. BOILED KOHLRABI.--Persons fond of kohlrabi...
        18       236       (14) Mention some methods of preparing vegetab...
9935    22       24        To add to both dry and cooked cereals protein ...
9936    20       106       1 pt. stewed onions 3 Tb. butter 3 Tb. flour 1...
9939    17       21        (20) What precautions should be observed in th...
9936    20       289       132. CORN, STRING BEANS, AND TOMATOES.--Those ...
9939    15       115       The _method of sealing_ must also be judged be...
53521   23       9         The head, tail, and fins of trout should not b...
9937    23       93        58. STEWED FRESH HERRING.--When fresh herring ...
9939    31       119                                           SERVING CANDY
        38       10        67. In all but the first of the recipes that f...
9936    4        18                                    FOODS CONTAINING MILK
        6        3         2. ECONOMICAL USE OF BUTTER.--In the home, but...
38464   52       1                                    Curried Calf's Brains.
9938    27       33                                                        E
        16       119       1/3 c. butter 2/3 c. sugar 4 egg yolks 1-1/4 c...
9935    19       7         33. So that the importance of these corn produ...

In [83]:
#doc_j.to_csv('jason_docs_table.csv')